In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 500)

In [3]:
ALL_EVALUATIONS="tests/evaluation_07_retrievers_000_009_sampled_50_queries_gpt-4-1106-preview_20231220.tsv"

BM25_RUN="tests/200-queries-passagens-by-BM25-pt_v2_with_query_id_run.txt"
BM25_E5_RUN="tests/bm25_e5_docs_000_999_50_sampled_queries_run.txt"
BM25_MT5_RUN="tests/200-queries-passagens-by-BM25-mT5_v4_with_query_id_run.txt"
SPLADE_RUN="tests/splade_pt_result_with_clueweb_id_run.txt"
COLBERTX_RUN="tests/colbertx_all_znormalized_validation_scores_000_199_run.txt"
E5_RUN="tests/e5_docs_000_009_50_sampled_queries_score_inv_run.txt"

In [33]:
E5_BASE="tests/e5_base_docs_000_009_50_sampled_queries.tsv"

In [5]:
retrievers_results = []

### Format GPT-4 evaluations to qrels format

In [6]:
eval_df = pd.read_csv(ALL_EVALUATIONS, sep='\t')

In [7]:
eval_df

,query_id,query,passage_id,mT5,splade,colbertx,bm25_e5,bm25,e5,e5_base,passage,score,reasoning,usage,cost,duration,saved_cost,e5 base
0,1,Qual a maior característica da fauna brasileira?,clueweb22-pt0000-82-13187_0,False,False,True,False,False,False,False,"Mamíferos do Brasil - Zoologia | Museu Nacional - UFRJ ➤ Início ➤ Exposições ➤ Zoologia (Acervo) ➤ Insetos ➤ Mamíferos do Brasil Mamíferos do Brasil Uma larga coleção de mamíferos da fauna brasileira. Entre os importantes representantes deste acervo de espécimes taxidermizados encontram-se grandes símbolos da diversidade biológica do Brasil. Bugio, Barbado, Guariba Bugio Cachorro-do-mato Caititu Capivara Caxinguelê, Serelepe Cervo-do-Pantanal Gato-do-Mato-pequeno Irara Lobo-Guará Lontra Mão-Pelada Muriqui Onça-Pintada Ouriço-Cacheiro, Cuandu Paca Preguiça-de-Bentinho Preguiça-de-três-dedos Rato-Coró Saruê, Gambá Sauá, Guigó Tamanduá-bandeira Tamanduá-í Tamanduá-mirim, meleta Tatu-Canastra Tatu-de-rabo-mole Tatu-Galinha Tatu-Peludo Veado Veado-mateiro « Voltar | Voltar ao Topo",1,"A passagem lista vários mamíferos da fauna brasileira, mas não descreve características gerais da fauna brasileira, como a pergunta solicita. A passagem está relacionada ao tema da fauna brasileira, mas não responde diretamente à pergunta sobre a maior característica da fauna.","{'prompt_tokens': 765, 'completion_tokens': 75, 'total_tokens': 840}",0.00000,6.693229,0.02745,NaN
1,1,Qual a maior característica da fauna brasileira?,clueweb22-pt0000-59-04599_2,False,False,True,False,False,False,False,"Segundo uma lista levantada pelo Comitê Brasileiro de Registros Ornitológicos (CBRO), algumas das principais espécies encontradas aqui no país são: Bem-te-vi, Arara-azul, Tuiuiú, Tucano e o Papagaio. Só no Cerrado pode-se encontrar por volta de 148 espécies de aves bem específicas da região. Algumas das aves que mais representam esta região brasileira são: Seriemas; Pavãozinho-do-pará; Coruja-orelhuda; Arara-canindé; Coruja-da-igreja; Arara-vermelha-grande; Tuiuiú (considerado o símbolo do Pantanal). Quando chegamos à região nordestina a ave brasileira já foi até tema de música, a famosa ave com seu voo incrível chamada “Asa branca”, ave que deu nome à música de mais sucesso na voz do cantor e compositor Luiz Gonzaga. A Amazônia é uma região brasileira que qualquer um se encanta pelas belezas naturais principalmente quando o assunto são espécies de aves. As araras com sua pelagem colorida chamam a atenção de todos e podem ser vistas por todos os lados na região. A segunda ave que pode ser encontrada facilmente na Amazônia é o tucano, com suas características marcantes, um bico longo e cores vibrantes que marcam a região dos olhos. Claro que podemos encontrar essas bela espécie de ave em zoológicos espalhados por todo o país ou em áreas de matas em geral.",1,"A passagem descreve várias espécies de aves encontradas no Brasil e suas características, mas não resume qual é a maior característica da fauna brasileira de forma geral. A pergunta pede uma característica ampla da fauna, enquanto a passagem foca em exemplos específicos de aves em diferentes regiões.","{'prompt_tokens': 860, 'completion_tokens': 82, 'total_tokens': 942}",0.00000,5.163547,0.03072,NaN
2,1,Qual a maior característica da fauna brasileira?,clueweb22-pt0000-64-06285_39,True,True,False,False,False,False,True,"A exuberante fauna e flora brasileiras, relatadas por Pero Vaz de Caminha em sua carta à Coroa Portuguesa, está sendo destruída gradativamente pela ação antrópica, por meio do desmatamento e da caça d... Leia redação completa Redação sem título. 02/07/2016 O Brasil tem a maior biodiversidade do mundo. Devido a sua variedade de espécies silvestres, desencadeou uma enorme ambição global ao tráfico de animais desde o período colonial. Atualmente, tal ato a... Leia redação completa Redação sem título. 19/09/2018 O Brasil com seu vasto território é um dos países com a maior biodiversidade de animais silvestres. Entretanto, não é dado a devida 

In [8]:
eval_df.drop_duplicates("query_id")[['query_id', 'query']]

,query_id,query
0,1,Qual a maior característica da fauna brasileira?
44,2,Por que os países Guiana e Suriname não são filiados a Conmebol?
96,9,quais países europeus seguem o regime monarquista?
140,11,quais os critérios de definição dos monumentos intitulados maravilhas do mundo moderno?
181,13,Como o Brasil reagiu a epidemia de AIDS no fim do século XX?
219,15,Qual a maior torcida de futebol do Brasil?
244,17,Quando se realizou o plebiscito popular para definir o sistema político do Brasil?
278,20,Por que a legislação de um país é tão importante?
336,21,Qual o nosso atual sistema de governo?
390,22,Qual o sistema de governo da África do Sul?


#### Check scores distribution on the current qrels

In [9]:
np.unique(eval_df['score'].to_numpy(), return_counts=True)

(array([0, 1, 2, 3]), array([946, 432, 379, 435]))

In [10]:
for group_name, group_df in eval_df.groupby('query_id'):
    print("\nquery_id={}".format(group_name))
    
    print(np.unique(group_df['score'].to_numpy(), return_counts=True))


query_id=1
(array([0, 1, 2, 3]), array([ 5, 21, 11,  7]))

query_id=2
(array([0, 1, 2, 3]), array([47,  2,  1,  2]))

query_id=9
(array([0, 1, 2, 3]), array([26,  4,  9,  5]))

query_id=11
(array([0, 1, 2, 3]), array([24,  8,  7,  2]))

query_id=13
(array([0, 1, 2, 3]), array([15,  8, 11,  4]))

query_id=15
(array([0, 1, 2, 3]), array([ 2, 10,  1, 12]))

query_id=17
(array([0, 1, 2, 3]), array([ 6,  2,  1, 25]))

query_id=20
(array([0, 1, 2, 3]), array([ 5, 31, 18,  4]))

query_id=21
(array([0, 1, 2, 3]), array([24, 15,  1, 14]))

query_id=22
(array([0, 1, 3]), array([29,  7,  5]))

query_id=24
(array([0, 1, 2, 3]), array([27, 17,  9,  2]))

query_id=26
(array([0, 1, 2, 3]), array([11,  2,  7, 20]))

query_id=28
(array([0, 1, 2, 3]), array([26,  7, 13,  4]))

query_id=47
(array([0, 1, 2]), array([25, 18, 15]))

query_id=49
(array([0, 1, 2, 3]), array([13, 14, 13,  1]))

query_id=51
(array([0, 1, 2, 3]), array([14, 13,  6,  1]))

query_id=54
(array([1, 2, 3]), array([ 7,  7, 27]))

que

In [11]:
eval_df.groupby('query_id').count()

,query,passage_id,mT5,splade,colbertx,bm25_e5,bm25,e5,e5_base,passage,score,reasoning,usage,cost,duration,saved_cost,e5 base
query_id,,,,,,,,,,,,,,,,,
1,44,44,44,44,44,44,44,44,39,44,44,44,44,44,44,44,5
2,52,52,52,52,52,52,52,52,43,52,52,52,52,52,52,52,9
9,44,44,44,44,44,44,44,44,41,44,44,44,44,44,44,44,3
11,41,41,41,41,41,41,41,41,32,41,41,41,41,41,41,41,9
13,38,38,38,38,38,38,38,38,34,38,38,38,38,38,38,38,4
15,25,25,25,25,25,25,25,25,24,25,25,25,25,25,25,25,1
17,34,34,34,34,34,34,34,34,31,34,34,34,34,34,34,34,3
20,58,58,58,58,58,58,58,58,51,58,58,58,58,58,58,58,7
21,54,54,54,54,54,54,54,54,50,54,54,54,54,54,54,54,4


#### Prepare the qrels format for trec_eval tools

In [12]:
eval_df['0'] = '0'

In [13]:
qrels_filename = os.path.join(os.path.dirname(ALL_EVALUATIONS), "{}_qrel.txt".format(os.path.splitext(os.path.basename(ALL_EVALUATIONS))[0]))

In [14]:
qrels_filename

'tests/evaluation_07_retrievers_000_009_sampled_50_queries_gpt-4-1106-preview_20231220_qrel.txt'

In [15]:
eval_df.sort_values('query_id')[['query_id', '0', 'passage_id', 'score']].to_csv(qrels_filename, header=None, index=False, sep=" ")

### Compute BM25 nDCG@10

In [16]:
result = !/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mndcg_cut.10 \
    {qrels_filename} tests/200-queries-passagens-by-BM25-pt_v2_with_query_id_run.txt

In [17]:
retrievers_results.append({'retriever':'BM25', 'nDCG@10': result[0].split('\t')[-1]})

In [18]:
retrievers_results

[{'retriever': 'BM25', 'nDCG@10': '0.4624'}]

### Compute BM25 + E5 nDCG@10

In [19]:
result = !/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mndcg_cut.10 \
    {qrels_filename} {BM25_E5_RUN}

In [20]:
retrievers_results.append({'retriever':'BM25+E5', 'nDCG@10': result[0].split('\t')[-1]})

### Compute BM25 + mT5 nDCG@10

In [21]:
result = !/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mndcg_cut.10 \
    {qrels_filename} {BM25_MT5_RUN}

In [22]:
retrievers_results.append({'retriever':'BM25+mT5', 'nDCG@10': result[0].split('\t')[-1]})

### Compute SPLADE nDCG@10

In [23]:
result = !/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mndcg_cut.10 \
    {qrels_filename} {SPLADE_RUN}

In [24]:
retrievers_results.append({'retriever':'SPLADE_pt-BR', 'nDCG@10': result[0].split('\t')[-1]})

### Compute ColBERTX nDCG@10

In [25]:
result = !/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mndcg_cut.10 \
    {qrels_filename} {COLBERTX_RUN}

In [26]:
retrievers_results.append({'retriever':'ColBERT-X_mMARCO_pt-BR', 'nDCG@10': result[0].split('\t')[-1]})

### Compute E5 nDCG@10

In [27]:
result = !/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mndcg_cut.10 \
    {qrels_filename} {E5_RUN}

In [28]:
retrievers_results.append({'retriever':'E5', 'nDCG@10': result[0].split('\t')[-1]})

### Prepare E5 base retrieval results in run format

In [34]:
e5_df = pd.read_csv(E5_BASE, sep="\t")

In [35]:
e5_df.groupby("query_id").head(10).count()

query          500
passage_id     500
score          500
query_id       500
score_inv      500
colbertx_id    500
passage        500
dtype: int64

In [36]:
e5_df

,query,passage_id,score,query_id,score_inv,colbertx_id,passage
0,Qual a maior característica da fauna brasileira?,clueweb22-pt0001-22-15255_0,0.122909,1,0.877091,5877599,"Fauna do Brasil - Toda Matéria Biologia › Zoologia Fauna do Brasil A fauna do Brasil é provavelmente a que possui a maior diversidade em espécies de animais vertebrados, anfíbios e primatas do globo terrestre. Existem três vezes mais espécies de peixes de água doce no Brasil do que em qualquer lugar no mundo. As aves também são muito variadas, colocando o Brasil como o país com a maior variedade de espécies na América do Sul e o terceiro do mundo em espécies de mamíferos. De igual modo, a Floresta Atlântica está entre as cinco regiões do planeta com mais espécies nativas, das quais 17% são aves e 10% são anfíbios e mamíferos encontrados somente na fauna brasileira. Vale citar que os animais da fauna brasileira são, via de regra, de pequeno porte. A importância científica da fauna brasileira é indescritível, pois contém a maior biodiversidade do mundo (cerca de 100 mil espécies de invertebrados, 8200 espécies vertebrados (713 mamíferos, 1826 aves, 721 répteis, 875 anfíbios, 2800 peixes continentais e 1300 peixes marinhos) das quais 627 já estão na lista negra das espécies ameaçadas de extinção (394 espécies terrestres e 233 espécies aquáticas)."
1,Qual a maior característica da fauna brasileira?,clueweb22-pt0000-96-00412_0,0.129059,1,0.870941,4583617,"15 ANIMAIS BRASILEIROS: Características e curiosidades Animais brasileiros: nativos, endêmicos e em risco de extinção Fauna quer dizer o conjunto de espécies que vivem em uma determinada região. Por isso, é importante saber diferenciar que quando falamos sobre a fauna brasileira, estamos falando sobre todas as espécies que habitam o Brasil, mas não necessariamente apenas de espécies endêmicas ou nativas, já que alguns animais são considerados espécies invasoras e/ou foram introduzidos pela pessoa humana. Para te mostrar um pouco da nossa incrível fauna, neste post do PeritoAnimal nos focamos em listar aqueles animais brasileiros: nativos e endêmicos que são icônicos na nossa história, além dos animais que só existem no Brasil. Continue lendo e encante-se com a magnitude de cada um deles! Também lhe pode interessar: 15 animais ameaçados de extinção no Brasil Índice Fauna brasileira Animais brasileiros Animais que só existem no Brasil 8 cachorros brasileiros Outros animais da fauna brasileira Fauna brasileira De acordo com o Instituto Chico Mendes, [1] o Brasil administra o maior patrimônio de biodiversidade do mundo. Em números isso se traduz em 120 mil espécies de invertebrados e 8930 espécies de vertebrados, aproximadamente, dentre eles são: 734 espécies de mamíferos;"
2,Qual a maior característica da fauna brasileira?,clueweb22-pt0001-90-10791_1,0.137393,1,0.862607,9091797,"Características dos \""corvos do Brasil\"" Os “corvos do Brasil” possuem características muito parecidas com toda a família Corvidae. Desde as características físicas, a inteligência dessas aves, a astúcia e a capacidade de ter empatia por seus colegas, veremos o que foi herdado e descobriremos o que as difere dos demais corvos. Acompanhe: ■ Características visuais Os animais do gênero corvidae costumam ser maiores e mais robustos com que outras espécies de aves. A maior parte das espécies de corvídeos possuem penas de cor escura, variando desde o vermelho acobreado, para o azul-marinho e o cinza. No entanto, as espécies mais conhecidas são mais aquelas que têm penas completamente pretas. Os corvídeos não costumam apresentar dimorfismo sexual. Possuem pernas fortes e envergadura um pouco mais avantajada se comparado a outras aves. ■ Distribuição e habitat Os corvídeos estão presentes em diversas regiões do Brasil, como florestas, cerrados e áreas urbanas. Se encaixam muito bem em diferentes locais e possuem uma alimentação diversificada que facilita a sua adaptação."
3,Qual a maior característica da fauna brasileira?,clueweb22-pt0000-77-0688

In [37]:
e5_run_df = e5_df.sort_values(["query_id", "score_inv"], ascending=[True, False]).groupby("query_id").head(10)

In [38]:
e5_run_df.shape

(500, 7)

In [39]:
e5_run_df['Q0'] = "Q0"
e5_run_df['comment'] = "gpt-4-1106-preview"
e5_run_df['index'] = np.tile(list(range(1, 11)), 50)

In [40]:
e5_run_df.head(12)

,query,passage_id,score,query_id,score_inv,colbertx_id,passage,Q0,comment,index
0,Qual a maior característica da fauna brasileira?,clueweb22-pt0001-22-15255_0,0.122909,1,0.877091,5877599,"Fauna do Brasil - Toda Matéria Biologia › Zoologia Fauna do Brasil A fauna do Brasil é provavelmente a que possui a maior diversidade em espécies de animais vertebrados, anfíbios e primatas do globo terrestre. Existem três vezes mais espécies de peixes de água doce no Brasil do que em qualquer lugar no mundo. As aves também são muito variadas, colocando o Brasil como o país com a maior variedade de espécies na América do Sul e o terceiro do mundo em espécies de mamíferos. De igual modo, a Floresta Atlântica está entre as cinco regiões do planeta com mais espécies nativas, das quais 17% são aves e 10% são anfíbios e mamíferos encontrados somente na fauna brasileira. Vale citar que os animais da fauna brasileira são, via de regra, de pequeno porte. A importância científica da fauna brasileira é indescritível, pois contém a maior biodiversidade do mundo (cerca de 100 mil espécies de invertebrados, 8200 espécies vertebrados (713 mamíferos, 1826 aves, 721 répteis, 875 anfíbios, 2800 peixes continentais e 1300 peixes marinhos) das quais 627 já estão na lista negra das espécies ameaçadas de extinção (394 espécies terrestres e 233 espécies aquáticas).",Q0,gpt-4-1106-preview,1
1,Qual a maior característica da fauna brasileira?,clueweb22-pt0000-96-00412_0,0.129059,1,0.870941,4583617,"15 ANIMAIS BRASILEIROS: Características e curiosidades Animais brasileiros: nativos, endêmicos e em risco de extinção Fauna quer dizer o conjunto de espécies que vivem em uma determinada região. Por isso, é importante saber diferenciar que quando falamos sobre a fauna brasileira, estamos falando sobre todas as espécies que habitam o Brasil, mas não necessariamente apenas de espécies endêmicas ou nativas, já que alguns animais são considerados espécies invasoras e/ou foram introduzidos pela pessoa humana. Para te mostrar um pouco da nossa incrível fauna, neste post do PeritoAnimal nos focamos em listar aqueles animais brasileiros: nativos e endêmicos que são icônicos na nossa história, além dos animais que só existem no Brasil. Continue lendo e encante-se com a magnitude de cada um deles! Também lhe pode interessar: 15 animais ameaçados de extinção no Brasil Índice Fauna brasileira Animais brasileiros Animais que só existem no Brasil 8 cachorros brasileiros Outros animais da fauna brasileira Fauna brasileira De acordo com o Instituto Chico Mendes, [1] o Brasil administra o maior patrimônio de biodiversidade do mundo. Em números isso se traduz em 120 mil espécies de invertebrados e 8930 espécies de vertebrados, aproximadamente, dentre eles são: 734 espécies de mamíferos;",Q0,gpt-4-1106-preview,2
2,Qual a maior característica da fauna brasileira?,clueweb22-pt0001-90-10791_1,0.137393,1,0.862607,9091797,"Características dos \""corvos do Brasil\"" Os “corvos do Brasil” possuem características muito parecidas com toda a família Corvidae. Desde as características físicas, a inteligência dessas aves, a astúcia e a capacidade de ter empatia por seus colegas, veremos o que foi herdado e descobriremos o que as difere dos demais corvos. Acompanhe: ■ Características visuais Os animais do gênero corvidae costumam ser maiores e mais robustos com que outras espécies de aves. A maior parte das espécies de corvídeos possuem penas de cor escura, variando desde o vermelho acobreado, para o azul-marinho e o cinza. No entanto, as espécies mais conhecidas são mais aquelas que têm penas completamente pretas. Os corvídeos não costumam apresentar dimorfismo sexual. Possuem pernas fortes e envergadura um pouco mais avantajada se comparado a outras aves. ■ Distribuição e habitat Os corvídeos estão presentes em diversas regiões do Brasil, como florestas, cerrados e áreas urbanas. Se encaixam muito bem em diferentes locais e possuem uma alimentação diversificada que facilita a sua adaptação.",Q0,gpt-4-1

In [41]:
e5_run_df[['query_id', 'Q0', "passage_id", "index", "score_inv", "comment"]].to_csv("tests/e5_base_docs_000_009_50_sampled_queries_score_inv_run.txt", 
                                                                                    header=None, index=False, sep=" ")

#### Compute E5 NDCG@10

In [42]:
result = !/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mndcg_cut.10 \
    {qrels_filename} "tests/e5_base_docs_000_009_50_sampled_queries_score_inv_run.txt"

In [43]:
retrievers_results.append({'retriever':'E5 base', 'nDCG@10': result[0].split('\t')[-1]})

### Check the overall results

In [44]:
pd.DataFrame(retrievers_results)

,retriever,nDCG@10
0,BM25,0.4624
1,BM25+E5,0.6589
2,BM25+mT5,0.7350
3,SPLADE_pt-BR,0.5982
4,ColBERT-X_mMARCO_pt-BR,0.6483
5,E5,0.5765
6,E5 base,0.4138
